In [1]:
import tabula

pdf_path = "./pdfs/2011.pdf"

# Read PDF and extract tables from pages 26 to 41
tables = tabula.read_pdf(pdf_path, pages='40-65', multiple_tables=True, stream=False)

# Save tables to a text file
output_file = "./2011.txt"
with open(output_file, 'w') as file:
    for table in tables:
        for row in table.itertuples(index=False):
            cleaned_row = []
            for cell in row:
                if isinstance(cell, str):
                    cleaned_cell = ' '.join(cell.split())
                    cleaned_row.append(cleaned_cell)
                else:
                    cleaned_row.append(cell)
            file.write('\t'.join(map(str, cleaned_row)) + '\n')


In [21]:
import re
import tabula

pdf_path = "./pdfs/2002.pdf"

# Read PDF and extract tables from pages 26 to 41
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, stream=False)

# Save tables to a text file
output_file = "./2002.txt"
with open(output_file, 'w', encoding='utf-8') as file:
    for table in tables:
        for row in table.itertuples(index=False):
            cleaned_row = []
            for cell in row:
                if isinstance(cell, str):
                    cleaned_cell = ' '.join(cell.split())
                    cleaned_row.append(cleaned_cell)
                else:
                    cleaned_row.append(cell)
            csv_contents = '\t'.join(map(str, cleaned_row)) + '\n'
            
            # Apply regex pattern to each row
            pattern = r'^(.*?),"([^"]+)",.*?,((?:\d+\.?\d*|),(?:\d+\.?\d*|),(?:\d+\.?\d*|),)'
            matches = re.findall(pattern, csv_contents, re.MULTILINE)
            
            if matches:
                # If matches are found, write them to the file
                for match in matches:
                    file.write('\t'.join(match) + '\n')
            else:
                # If no matches are found, write the original row to the file
                file.write(csv_contents)


In [32]:
import re

def convert_to_tsv(content):
    # Pattern for the headers
    header_pattern = r"Employer Name City State HRC Rating"
    
    # Pattern for the lines with company data
    company_pattern = r"^(.*\S)\s+([A-Za-z\s-]+?)\s+([A-Z]{2})\s+(\d+)$"
    
    tsv_lines = []
    removed_lines = []
    
    for line in content.split("\n"):
        if re.match(header_pattern, line):
            # Skip headers
            continue
        
        match = re.match(company_pattern, line)
        if match:
            tsv_line = "\t".join(match.groups())
            tsv_lines.append(tsv_line)
        else:
            # Collect lines that did not match
            removed_lines.append(line)
    
    tsv_content = "\n".join(tsv_lines)
    
    # Add removed lines to the end with a special note
    if removed_lines:
        tsv_content += "\n\n##Some rows did not match the pattern:\n"
        tsv_content += "\n".join(removed_lines)
    
    return tsv_content

# Read from the input file
with open("2002.tsv", "r") as file:
    content = file.read()

# Convert the content to TSV format
tsv_content = convert_to_tsv(content)

# Write to the output file
with open("output.tsv", "w") as file:
    file.write(tsv_content)
